## Query Logging을 위한 설정
1. Query Logging을 위한 Index 생성
2. Query Logging을 위한 Function 생성
3. Query Logging테스트

In [3]:
## aoss 정의

from opensearchpy import OpenSearch
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

def getOpenSearchClient(endpoint):
    
    session = boto3.Session()
    region = session.region_name
    service = 'aoss'
    credentials = session.get_credentials()
    awsauth = AWS4Auth(credentials.access_key, credentials.secret_key,
                   region, service, session_token=credentials.token)
    
    aoss_client = OpenSearch(
        hosts=[{'host': endpoint, 'port': 443}],
        http_auth=awsauth,
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection,
        timeout=6000
    )
    return aoss_client

def deleteIndex(client, index_name):
    # client = getOpenSearchClient()
    response = client.indices.delete(index_name)
    print('\nDeleting index:')
    print(response)
    
    return response

# index 생성 함수
def createIndex(client, index_name, index_schema=None):    
    # client = getOpenSearchClient()

    if index_schema:
        response = client.indices.create(index_name, body=index_schema)
    else:
        response = client.indices.create(index_name)
    print('\nCreating index:')
    print(response)
    
    return response


## index 생성

In [6]:
## 초기 index 생성
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()
aoss_endpoint = os.getenv("AOSS_ENDPOINT")
# aoss_collection=os.getenv("AOSS_COLLECTION")
# aoss_vector_index=os.getenv("AOSS_VECTOR_INDEX")
aoss_chatlog_index = os.getenv("AOSS_CHATLOG_INDEX")

# aoss client 생성
aoss_client = getOpenSearchClient(aoss_endpoint)

# 인덱스 생성

# 챗로그를 위한 인덱스 스키마
chatlog_index_schema = {
    "settings": {
        "index": {
            "number_of_shards": 1,
            "number_of_replicas": 1
        }
    },
    "mappings": {
        "properties": {
            "timestamp": {"type": "date"},
            "session_id": {"type": "keyword"},
            "user_id": {"type": "keyword"},
            "question": {"type": "text", "analyzer": "standard"},
            "answer": {"type": "text", "analyzer": "standard"},
            "metadata": {"type": "object"}
        }
    }
}

# OpenSearch 클라이언트 생성
aoss_client = getOpenSearchClient(aoss_endpoint)

# 챗로그 인덱스 생성
deleteIndex(aoss_client, aoss_chatlog_index)
createIndex(aoss_client, aoss_chatlog_index, chatlog_index_schema)



Deleting index:
{'acknowledged': True}

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'rag-nice-chatlog-index'}


{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'rag-nice-chatlog-index'}

### 로깅 테스트 

In [5]:
import datetime
import uuid

## 초기 index 생성
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()
aoss_endpoint = os.getenv("AOSS_ENDPOINT")
# aoss_collection=os.getenv("AOSS_COLLECTION")
# aoss_vector_index=os.getenv("AOSS_VECTOR_INDEX")
aoss_chatlog_index = os.getenv("AOSS_CHATLOG_INDEX")

# aoss client 생성
aoss_client = getOpenSearchClient(aoss_endpoint)

def log_chat(client, index_name, user_id, question, answer, metadata=None):
    doc = {
        "timestamp": datetime.datetime.now().isoformat(),
        "session_id": str(uuid.uuid4()),
        "user_id": user_id,
        "question": question,
        "answer": answer,
        "metadata": metadata or {}
    }
    
    response = client.index(
        index=index_name,
        body=doc,
    )
    
    print(f"Chat logged: {response['result']}")
    return response

# 사용 예시
user_id = "user123"
question = "인공지능이란 무엇인가요?"
answer = "인공지능은 인간의 학습능력, 추론능력, 지각능력을 인공적으로 구현한 컴퓨터 프로그램 또는 이를 포함한 컴퓨터 시스템을 말합니다."
metadata = {"model": "Cladue 3", "confidence": 0.95}

log_chat(aoss_client, aoss_chatlog_index, user_id, question, answer, metadata)

Chat logged: created


{'_index': 'rag-nice-chatlog-index',
 '_id': '1%3A0%3AR3kdIJEB_u-E-vvE5Szr',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 0, 'successful': 0, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 0}

In [11]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3
from dotenv import load_dotenv
import os

# .env 파일 로드 (이미 로드되었다면 생략 가능)
load_dotenv()
aoss_endpoint = os.getenv("AOSS_ENDPOINT")
aoss_chatlog_index = os.getenv("AOSS_CHATLOG_INDEX")


def search_chat_logs(query=None, user_id=None, start_date=None, end_date=None, size=10):
    client = getOpenSearchClient(aoss_endpoint)
    index_name = aoss_chatlog_index
    """
    채팅 로그를 검색하는 함수
    
    :param client: OpenSearch 클라이언트
    :param index_name: 검색할 인덱스 이름
    :param query: 검색할 키워드 (질문 또는 답변에서 검색)
    :param user_id: 특정 사용자의 채팅만 검색
    :param start_date: 검색 시작 날짜 (ISO 형식의 문자열, 예: '2023-01-01T00:00:00')
    :param end_date: 검색 종료 날짜 (ISO 형식의 문자열, 예: '2023-12-31T23:59:59')
    :param size: 반환할 결과의 최대 개수
    :return: 검색 결과
    """
    must_conditions = []

    if query:
        must_conditions.append({
            "multi_match": {
                "query": query,
                "fields": ["question", "answer"]
            }
        })

    if user_id:
        must_conditions.append({"term": {"user_id": user_id}})

    if start_date or end_date:
        date_range = {}
        if start_date:
            date_range["gte"] = start_date
        if end_date:
            date_range["lte"] = end_date
        must_conditions.append({"range": {"timestamp": date_range}})

    body = {
        "query": {
            "bool": {
                "must": must_conditions
            }
        },
        "sort": [{"timestamp": "desc"}],
        "size": size
    }

    response = client.search(index=index_name, body=body)

    return response['hits']['hits']

# 사용 예시
def print_chat_logs(logs):
    for log in logs:
        source = log['_source']
        print(f"Timestamp: {source['timestamp']}")
        print(f"User ID: {source['user_id']}")
        print(f"Question: {source['question']}")
        print(f"Answer: {source['answer']}")
        print(f"Metadata: {source['metadata']}")
        print("-" * 50)

# 모든 로그 검색
all_logs = search_chat_logs()
print("All logs:")
print_chat_logs(all_logs)

# 특정 사용자의 로그 검색
user_logs = search_chat_logs(user_id="user123")
print("\nLogs for user123:")
print_chat_logs(user_logs)

# 특정 키워드로 로그 검색
keyword_logs = search_chat_logs(query="경쟁지역")
print("\nLogs containing '경쟁지역':")
print_chat_logs(keyword_logs)

# 특정 날짜 범위의 로그 검색
date_logs = search_chat_logs(
                             start_date="2024-01-01T00:00:00", 
                             end_date="2024-12-31T23:59:59")
print("\nLogs from 2024:")
print_chat_logs(date_logs)

All logs:
Timestamp: 2024-08-05T14:15:31.389856
User ID: user123
Question: 삼성2동은 성장유형이 어때?
Answer: 제공된 정보에 따르면, 삼성2동은 성장유형이 '경쟁지역(4사분면) - 침체지속'으로 분류되어 있습니다. 이는 삼성2동의 일식 업종에서 매출 성장률은 낮지만 점포수 증가율은 다소 높은 상황임을 의미합니다. 즉, 매출은 감소하고 있지만 점포수는 늘어나고 있어 경쟁이 심화되고 있는 지역으로 해석할 수 있습니다.
Metadata: {'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'run_id': 'run-393cbb4c-0912-4513-986e-a848ad67efb6-0'}
--------------------------------------------------

Logs for user123:
Timestamp: 2024-08-05T14:15:31.389856
User ID: user123
Question: 삼성2동은 성장유형이 어때?
Answer: 제공된 정보에 따르면, 삼성2동은 성장유형이 '경쟁지역(4사분면) - 침체지속'으로 분류되어 있습니다. 이는 삼성2동의 일식 업종에서 매출 성장률은 낮지만 점포수 증가율은 다소 높은 상황임을 의미합니다. 즉, 매출은 감소하고 있지만 점포수는 늘어나고 있어 경쟁이 심화되고 있는 지역으로 해석할 수 있습니다.
Metadata: {'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'run_id': 'run-393cbb4c-0912-4513-986e-a848ad67efb6-0'}
--------------------------------------------------

Logs containing '경쟁지역':
Timestamp: 2024-08-05T14:15:31.389856
User ID: user123
Question: 삼성2동은 성장유형이 어